# MNIST
ref: https://www.tc3.co.jp/jaxflax-introduction-with-mnist/  
ref: https://tech.morikatron.ai/entry/2020/12/07/100000

In [72]:
from tqdm import tqdm
import time

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import jax
import jax.numpy as jnp
from jax import random
from jax import jit

from flax import linen as nn
from flax.training import train_state

In [77]:

# 順伝播の計算
@jit
def mlp_predict(params, x):
    w, b = params[0]
    x = jnp.dot(x, w) + b
    for w, b in params[1:]:
        x = jnp.maximum(0, x)
        x = jnp.dot(x, w) + b
    y = softmax(x)
    return y
def softmax(x):
    x = x - jnp.max(x, axis=1, keepdims=True)  # eの階乗でオーバーフローを起こさないための工夫です。
    return jnp.exp(x) / jnp.sum(jnp.exp(x), axis=1, keepdims=True)

def mlp_loss(params, x, y):
    probs = mlp_predict(params, x)
    loss = jnp.mean(cross_entropy(probs, y))
    return loss
def cross_entropy(x, y):
    return jnp.sum(- y * jnp.log(x + 1e-8), axis=1)

# パラメータの初期値を得る
def init_mlp_params(input_size: int, output_size: int, num_units: list, seed: int = 0):
    params = []
    num_units.append(output_size)
    key = random.PRNGKey(seed)
    last_out = input_size
    for unit in num_units:
        key, subkey = random.split(key)
        # using He initialization method
        x, w = random.normal(key, (last_out, unit), dtype=jnp.float32) * jnp.sqrt(2 / unit), random.normal(subkey, (unit, ), dtype=jnp.float32) * jnp.sqrt(2 / unit)
        params.append((x, w))
        last_out = unit
    return params

# SGDを用いた勾配降下
@jit
def apply_grads(params, grads, lr=0.001):
    return [(w - w_grad * lr, b - b_grad * lr)
            for (w, b), (w_grad, b_grad) in zip(params, grads)]
    

def train_one_epoch(params, X_train, y_train, epoch, lr, bch_size):
    num_samples = X_train.shape[0]
    random_sample_idx = random.permutation(random.PRNGKey(epoch), jnp.arange(num_samples))
    print(random_sample_idx)
    for idx in tqdm(range(0, num_samples, bch_size)):
        mini_batch_idx = random_sample_idx[idx:idx+bch_size]
        print(mini_batch_idx)
        mini_batch_x = X_train[mini_batch_idx]
        mini_batch_y = y_train[mini_batch_idx]
        params_grad = grad(mlp_loss, argnums=0)(params, mini_batch_x, mini_batch_y)
        params = apply_grads(params, params_grad, lr)
        loss = mlp_loss(params, mini_batch_x, mini_batch_y)
    return loss, params

In [78]:
X_train[1]

KeyError: 1

In [67]:
learning_rate = 0.01
batch_size = 256
num_units = [512, 512]
num_epochs = 20
seed = 1234

X,y = fetch_openml('mnist_784', return_X_y=True)


In [68]:
one_hot_encoder = OneHotEncoder()
yy = one_hot_encoder.fit_transform(y.to_numpy().reshape(-1, 1)).A 

X_train, X_test, y_train, y_test = train_test_split(X / 255.0, yy, random_state=seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

print(X_train.shape)
print(y_train.shape)

(42000, 784)
(42000, 10)


In [76]:
params = init_mlp_params(X_train.shape[1], y_train.shape[1], num_units, seed)

for i in range(num_epochs):
    start_time = time.time()
    training_loss, params = train_one_epoch(params, X_train, y_train, i, learning_rate, batch_size)
    validation_loss = validation(params, X_valid, y_valid)
    epoch_time = time.time() - start_time
    print(f"EPOCH: {i} time: {epoch_time:.3f} training_loss: {training_loss:.3f} validation_loss: {validation_loss:.3f}")
#--

[25780 16732 33318 ... 34154   734 16604]


  0%|          | 0/165 [00:00<?, ?it/s]

[25780 16732 33318  6379 33123  8095  3252  6260 19099  2108  1195 26789
 20728 13898 24836 26820 36283 20129   900 18587 17077 14865 29853 31008
 39916   327  4718   131  5549 30734 30198 29375  1917 13688 33823 36845
 22201  7909  8920  6083  6962 19377 25361 16995 30337 36245  9485 28103
 12690 17329  5065 28016 11597   105  1398 22990  5875 35047 38192 10281
  7633 30769 13648 24370 17554 19604 24875 33043  8077 27592 31276 11892
 26400 21389 12609  4693 22113 30001 38522  6632 14218 18554  4141 12840
  5865 23497 41647 19710 41886 41395 20992 24496 11359 37652 19581 17073
 34278 11207 22220 28499  7232  7678 15337 14328  1549  7830 33411  3301
 19449 37901  7385 30682 32155 32221 28756 13962 30404  8215 38841 17627
 17752 36308 37438   278 41682 37725 37665 21186  4819  2552 33799 28222
 26875 31352  1289  1602 38447 32036   401 32589 16644 18116 40086 21657
 13095 14683 19857 33254  8046  8891 39151  6437 31813 33741 16865 12545
  5778  5040 19236 32526 31584 24533 21200 30154 26

KeyError: "None of [Int64Index([25780, 16732, 33318,  6379, 33123,  8095,  3252,  6260, 19099,\n             2108,\n            ...\n            36892, 14065, 15677, 21443, 38512,   526, 31274,  8861,  5617,\n             1669],\n           dtype='int64', length=256)] are in the [columns]"

In [ ]:

#accuracy = compute_accuracy(params, X_test, y_test)
#print(f"training finished. test acc: {accuracy:.4f}")